In [1]:
library("caper")

Loading required package: ape
Warning message:
: package ‘ape’ was built under R version 3.2.5Loading required package: MASS
Loading required package: mvtnorm


# Analysis of growth rate dataset

**Read in the datasets**

In [179]:
#########Growth rate analysis
data_table <-read.csv('../Data/growth_rate_dataset_10_2017_EXTENDED.csv', sep='\t')
# data_table <-read.csv('../Data/growth_rate_dataset_10_2017_EXTENDED_NO_BACTEROIDETES.csv', sep='\t')
data_tree <-read.tree('../Data/growth_rates_RAxML.newick')


####Test 2 more metrics
data_table['I_diff_rp'] <- data_table['Information_content_ribo..20_.4.'] - data_table['Information_content_all..20_.4.']
data_table['deltaI_diff_rp'] <- data_table['SD_summary_ribo_info'] - data_table['SD_summary_all_info']



data_comparative <- comparative.data(phy = data_tree, data = data_table,
                                    names.col = X.Id, vcv = FALSE, na.omit = FALSE, warn.dropped = TRUE)

In [180]:
colnames(data_table)

[1] "X.Id"                                        
 [2] "Organism"                                    
 [3] "TaxonID"                                     
 [4] "Full.Taxonomy"                               
 [5] "Doubling.Time"                               
 [6] "CDS_GC"                                      
 [7] "CDS_GC_nostartstop"                          
 [8] "Ribo_prot_GC_nostartstop"                    
 [9] "CDS_number"                                  
[10] "Intragenic_GC"                               
[11] "Gene_density"                                
[12] "Coding_density"                              
[13] "Genome_length"                               
[14] "Intragenic_length"                           
[15] "CDS_length"                                  
[16] "tRNA_number"                                 
[17] "All_NC_wright"                               
[18] "All_NC_novembre"                             
[19] "All_NC_info"                                 
[20] "Ribo_NC_wright"                              
[21] "Ribo_NC_novembre"                            
[22] "Ribo_NC_info"                                
[23] "Ribo_prot_number"                            
[24] "Information_content_all..20_.4."             
[25] "Information_content_ribo..20_.4."            
[26] "Information_content_all_mean_random..20_.4." 
[27] "Information_content_ribo_mean_random..20_.4."
[28] "Mean_binding_all"                            
[29] "Mean_binding_ribo"                           
[30] "Mean_binding_all_mean_random"                
[31] "Mean_binding_ribo_mean_random"               
[32] "Location_of_best_binding_all"                
[33] "Location_of_best_binding_ribo"               
[34] "ATG_start_percent"                           
[35] "X16s_copies"                                 
[36] "X23s_copies"                                 
[37] "CDS_with_SD_utr_motif"                       
[38] "Expected_CDS_with_SD_utr_motif"              
[39] "CDS_with_SD_utr_binding"                     
[40] "Expected_CDS_with_SD_utr_binding"            
[41] "RP_information_percentile"                   
[42] "RP_information_zscore"                       
[43] "aSD_binding_score_ribosome"                  
[44] "aSD_binding_score_all"                       
[45] "Internal_SD_seqs"                            
[46] "Mean_folding_init"                           
[47] "Mean_folding_internal"                       
[48] "Mean_folding_diff"                           
[49] "Mean_folding_init_ribo"                      
[50] "Mean_folding_internal_ribo"                  
[51] "Mean_folding_diff_ribo"                      
[52] "aSD_binding_summary"                         
[53] "folding_energy_init_summary"                 
[54] "folding_energy_diff_summary"                 
[55] "NC_summary_info"                             
[56] "NC_summary_novembre"                         
[57] "SD_summary_all_info"                         
[58] "SD_summary_ribo_info"                        
[59] "SD_summary_all_motif_diff"                   
[60] "SD_summary_all_binding_diff"                 
[61] "I_diff_rp"                                   
[62] "deltaI_diff_rp"

**Investigate individual predictors**

In [185]:
model.pgls <- pgls(tRNA_numbers ~ 1,
                   data = data_comparative, lambda='ML')
summary(model.pgls)


Call:
pgls(formula = X16s_copies ~ 1, data = data_comparative, lambda = "ML")

Residuals:
     Min       1Q   Median       3Q      Max 
-10.8725  -2.4897  -0.2718   2.1558   9.8883 

Branch length transformations:

kappa  [Fix]  : 1.000
lambda [ ML]  : 0.983
   lower bound : 0.000, p = < 2.22e-16
   upper bound : 1.000, p = < 2.22e-16
   95.0% CI   : (0.964, 0.992)
delta  [Fix]  : 1.000

Coefficients:
            Estimate Std. Error t value Pr(>|t|)   
(Intercept)  2.74680    0.83786  3.2784 0.001246 **
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 3.562 on 186 degrees of freedom
Multiple R-squared:     0,	Adjusted R-squared:     0 
F-statistic:   NaN on 0 and 186 DF,  p-value: NA 

**Calculate and write results data table for entire list of predictors that includes the combined model R2, individual R2 for each predictor, and the R2 for the combined model without each individual predictor**

A quirk to note here: from time to time there can be an error in caper's PGLS optimization function. My hack is just to slightly adjust the bounds on the lambda (i.e. from 1e-05 to 1e-04) and it usually works.

Also worth noting is that I don't use R frequently. If it's not obvious, apologies on the poor code below that nevertheless gets the job done.

In [69]:
# ### SD using deltaI
# metrics <- c('SD_summary_all_info', 'ATG_start_percent', 'NC_summary_info', 
#              'tRNA_number', 'X16s_copies', 'aSD_binding_summary', 'Mean_folding_diff')
# out_file_name <- '../Results/growth_rate_correlation_data.csv'

#### SD using binding
# metrics <- c('SD_summary_all_binding_diff', 'ATG_start_percent', 'NC_summary_info', 
#              'tRNA_number', 'X16s_copies', 'aSD_binding_summary', 'Mean_folding_diff')
# out_file_name <- '../Results/growth_rate_correlation_data_binding.csv'

#### SD in ribosomal proteins only
metrics <- c('SD_summary_ribo_info', 'ATG_start_percent', 'NC_summary_info', 
             'tRNA_number', 'X16s_copies', 'aSD_binding_summary', 'Mean_folding_diff')
out_file_name <- '../Results/growth_rate_correlation_data_riboprots.csv'

#### SD in ribosomal proteins diff
# metrics <- c('I_diff_rp', 'ATG_start_percent', 'NC_summary_info', 
#              'tRNA_number', 'X16s_copies', 'aSD_binding_summary', 'Mean_folding_diff')
# metrics <- c('deltaI_diff_rp', 'ATG_start_percent', 'NC_summary_info', 
#              'tRNA_number', 'X16s_copies', 'aSD_binding_summary', 'Mean_folding_diff')

In [70]:
rsq_table <- data.frame(matrix(ncol=8, nrow=8))
colnames(rsq_table) <- c('Individual(R2)', 'Individual(Lambda)',
                         'Individual(Lambda CI lower)', 'Individual(Lambda CI higher)',
                         'Without(R2)', 'Without(Lambda)',
                         'Without(Lambda CI lower)', 'Without(Lambda CI higher)')
rownames(rsq_table) <- c(c("Full_model", metrics))

temp <- paste(metrics, collapse = ' + ')
model.pgls <- pgls(as.formula(paste("log(Doubling.Time) ~", temp)),
               data = data_comparative, lambda='ML')
print(paste("log(Doubling.Time) ~", temp, "rsq", summary(model.pgls)$r.squared))
rsq_table["Full_model", 1] <- summary(model.pgls)$r.squared
rsq_table["Full_model", 2] <- summary(model.pgls)$param[2]
rsq_table["Full_model", 3] <- summary(model.pgls)$param.CI$lambda$ci.val[1]
rsq_table["Full_model", 4] <- summary(model.pgls)$param.CI$lambda$ci.val[2]
rsq_table["Full_model", 5] <- ''
rsq_table["Full_model", 6] <- ''
rsq_table["Full_model", 7] <- ''
rsq_table["Full_model", 8] <- ''

for(i in metrics){
    model.pgls <- pgls(as.formula(paste("log(Doubling.Time) ~", i)),
                   data = data_comparative, lambda='ML')
    print(paste("log(Doubling.Time ~", i, "rsq", summary(model.pgls)$r.squared))
    rsq_table[i, 1] <- summary(model.pgls)$r.squared
    rsq_table[i, 2] <- summary(model.pgls)$param[2]
    rsq_table[i, 3] <- summary(model.pgls)$param.CI$lambda$ci.val[1]
    rsq_table[i, 4] <- summary(model.pgls)$param.CI$lambda$ci.val[2]

    temp_metrics <- metrics[metrics != i];
    temp <- paste(temp_metrics, collapse = ' + ')
    model.pgls <- pgls(as.formula(paste("log(Doubling.Time) ~", temp)),
                   data = data_comparative, lambda='ML')
    print(paste("log(Doubling.Time) ~", temp, "rsq", summary(model.pgls)$r.squared))
    rsq_table[i, 5] <- summary(model.pgls)$r.squared
    rsq_table[i, 6] <- summary(model.pgls)$param[2]
    rsq_table[i, 7] <- summary(model.pgls)$param.CI$lambda$ci.val[1]
    rsq_table[i, 8] <- summary(model.pgls)$param.CI$lambda$ci.val[2]
}

[1] "log(Doubling.Time) ~ SD_summary_ribo_info + ATG_start_percent + NC_summary_info + tRNA_number + X16s_copies + aSD_binding_summary + Mean_folding_diff rsq 0.267102435382499"
[1] "log(Doubling.Time ~ SD_summary_ribo_info rsq 0.0395560406508757"
[1] "log(Doubling.Time) ~ ATG_start_percent + NC_summary_info + tRNA_number + X16s_copies + aSD_binding_summary + Mean_folding_diff rsq 0.23083969931211"
[1] "log(Doubling.Time ~ ATG_start_percent rsq 0.0195794612234714"
[1] "log(Doubling.Time) ~ SD_summary_ribo_info + NC_summary_info + tRNA_number + X16s_copies + aSD_binding_summary + Mean_folding_diff rsq 0.248396283345188"
[1] "log(Doubling.Time ~ NC_summary_info rsq 0.173725409536581"
[1] "log(Doubling.Time) ~ SD_summary_ribo_info + ATG_start_percent + tRNA_number + X16s_copies + aSD_binding_summary + Mean_folding_diff rsq 0.172892742722205"
[1] "log(Doubling.Time ~ tRNA_number rsq 0.0609079114942802"
[1] "log(Doubling.Time) ~ SD_summary_ribo_info + ATG_start_percent + NC_summary_info + X

In [71]:
rsq_table

,Individual(R2),Individual(Lambda),Individual(Lambda CI lower),Individual(Lambda CI higher),Without(R2),Without(Lambda),Without(Lambda CI lower),Without(Lambda CI higher)
Full_model,0.2671024,0.9409209,0.8643556,0.9753925,,,,
SD_summary_ribo_info,0.03955604,0.9764558,0.9436135,0.9916363,0.23083969931211,0.955499403012802,0.90326254896897,0.981203890380381
ATG_start_percent,0.01957946,0.9782529,0.9513469,0.9919281,0.248396283345188,0.949200701537513,0.881780426769595,0.979385299037689
NC_summary_info,0.1737254,0.9641915,0.9167162,0.9865308,0.172892742722205,0.962321992204807,0.916085949043635,0.984432829535443
tRNA_number,0.06090791,0.9789503,0.9495268,0.9927643,0.256187535682464,0.944926819960872,0.875360540324111,0.976899402082156
X16s_copies,0.05594219,0.9797408,0.9515697,0.992931,0.261210226081695,0.944175644491417,0.873135679876011,0.976570031983492
aSD_binding_summary,0.07601171,0.97724,0.9477932,0.9914256,0.259047050018651,0.944631570775115,0.873449223800992,0.976985982827197
Mean_folding_diff,0.07806029,0.9767075,0.950233,0.9902538,0.232659534678406,0.946142740254226,0.871955256971015,0.978904691631479


In [21]:
# write.table(rsq_table,file=out_file_name)

**Write model/s for plotting**

In [22]:
###Just the deltaI model
model.pgls <- pgls(log(Doubling.Time) ~ SD_summary_all_info,
                   data = data_comparative, lambda='ML',
                   bounds=list(lambda=c(1e-5,0.9999), kappa=c(1e-6,3), delta=c(1e-6,3)))
summary(model.pgls)
write.csv(model.pgls$fitted, '../Results/growth_fitted_model_SDinfo.csv')


Call:
pgls(formula = log(Doubling.Time) ~ SD_summary_all_info, data = data_comparative, 
    lambda = "ML", bounds = list(lambda = c(1e-05, 0.9999), kappa = c(1e-06, 
        3), delta = c(1e-06, 3)))

Residuals:
    Min      1Q  Median      3Q     Max 
-5.6858 -1.5022 -0.1795  1.0408  3.8776 

Branch length transformations:

kappa  [Fix]  : 1.000
lambda [ ML]  : 0.970
   lower bound : 0.000, p = < 2.22e-16
   upper bound : 1.000, p = 3.8144e-09
   95.0% CI   : (0.931, 0.989)
delta  [Fix]  : 1.000

Coefficients:
                    Estimate Std. Error t value  Pr(>|t|)    
(Intercept)          2.97000    0.43890  6.7669 1.682e-10 ***
SD_summary_all_info -0.95263    0.20199 -4.7162 4.721e-06 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.861 on 185 degrees of freedom
Multiple R-squared: 0.1073,	Adjusted R-squared: 0.1025 
F-statistic: 22.24 on 1 and 185 DF,  p-value: 4.721e-06 

In [33]:
###The full model using deltaI
model.pgls <- pgls(log(Doubling.Time) ~ SD_summary_all_info + ATG_start_percent + NC_summary_info +
                   tRNA_number + X16s_copies + aSD_binding_summary + Mean_folding_diff,
                   data = data_comparative, lambda='ML',
                   bounds=list(lambda=c(1e-5,0.9999), kappa=c(1e-6,3), delta=c(1e-6,3)))
summary(model.pgls)
# write.csv(model.pgls$fitted, '../Results/growth_fitted_model_full.csv')


Call:
pgls(formula = log(Doubling.Time) ~ SD_summary_all_info + ATG_start_percent + 
    NC_summary_info + tRNA_number + X16s_copies + aSD_binding_summary + 
    Mean_folding_diff, data = data_comparative, lambda = "ML", 
    bounds = list(lambda = c(1e-05, 0.9999), kappa = c(1e-06, 
        3), delta = c(1e-06, 3)))

Residuals:
    Min      1Q  Median      3Q     Max 
-3.3481 -1.0018 -0.1972  0.7106  5.4108 

Branch length transformations:

kappa  [Fix]  : 1.000
lambda [ ML]  : 0.913
   lower bound : 0.000, p = 1.07e-07
   upper bound : 1.000, p = 1.0214e-14
   95.0% CI   : (0.797, 0.963)
delta  [Fix]  : 1.000

Coefficients:
                      Estimate Std. Error t value  Pr(>|t|)    
(Intercept)          4.8505252  1.1430637  4.2434 3.527e-05 ***
SD_summary_all_info -0.9561017  0.1891738 -5.0541 1.062e-06 ***
ATG_start_percent   -0.9122341  1.2054638 -0.7567  0.450195    
NC_summary_info     15.8161239  2.8148155  5.6189 7.249e-08 ***
tRNA_number         -0.0070862  0.0080560 -0.

# Analysis of phylogenetic co-variation in larger dataset

**Read in the data**

In [186]:
data_table <-read.csv('../Data/Pruned_df_Domain_Bacteria_10_2017.csv', sep='\t')
data_tree <-read.tree('../Data/Pruned_tree_Domain_Bacteria_10_2017.newick')

data_comparative <- comparative.data(phy = data_tree, data = data_table,
                                    names.col = X.Id, vcv = FALSE, na.omit = FALSE, warn.dropped = TRUE)

In [187]:
colnames(data_table)

[1] "X.Id"                                        
 [2] "Organism"                                    
 [3] "TaxonID"                                     
 [4] "Full_Taxonomy"                               
 [5] "Banfield_tree_name"                          
 [6] "CDS_GC"                                      
 [7] "CDS_GC_nostartstop"                          
 [8] "Ribo_prot_GC_nostartstop"                    
 [9] "CDS_number"                                  
[10] "Intragenic_GC"                               
[11] "Gene_density"                                
[12] "Coding_density"                              
[13] "Genome_length"                               
[14] "Intragenic_length"                           
[15] "CDS_length"                                  
[16] "tRNA_number"                                 
[17] "All_NC_wright"                               
[18] "All_NC_novembre"                             
[19] "All_NC_info"                                 
[20] "Ribo_NC_wright"                              
[21] "Ribo_NC_novembre"                            
[22] "Ribo_NC_info"                                
[23] "Ribo_prot_number"                            
[24] "Information_content_all..20_.4."             
[25] "Information_content_ribo..20_.4."            
[26] "Information_content_all_mean_random..20_.4." 
[27] "Information_content_ribo_mean_random..20_.4."
[28] "Mean_binding_all"                            
[29] "Mean_binding_ribo"                           
[30] "Mean_binding_all_mean_random"                
[31] "Mean_binding_ribo_mean_random"               
[32] "Location_of_best_binding_all"                
[33] "Location_of_best_binding_ribo"               
[34] "ATG_start_percent"                           
[35] "X16s_copies"                                 
[36] "X23s_copies"                                 
[37] "CDS_with_SD_utr_motif"                       
[38] "Expected_CDS_with_SD_utr_motif"              
[39] "CDS_with_SD_utr_binding"                     
[40] "Expected_CDS_with_SD_utr_binding"            
[41] "RP_information_percentile"                   
[42] "RP_information_zscore"                       
[43] "aSD_binding_score_ribosome"                  
[44] "aSD_binding_score_all"                       
[45] "Internal_SD_seqs"                            
[46] "Mean_folding_init"                           
[47] "Mean_folding_internal"                       
[48] "Mean_folding_diff"                           
[49] "Mean_folding_init_ribo"                      
[50] "Mean_folding_internal_ribo"                  
[51] "Mean_folding_diff_ribo"                      
[52] "aSD_binding_summary"                         
[53] "folding_energy_init_summary"                 
[54] "folding_energy_diff_summary"                 
[55] "NC_summary_info"                             
[56] "NC_summary_novembre"                         
[57] "SD_summary_all_info"                         
[58] "SD_summary_ribo_info"                        
[59] "SD_summary_all_motif_diff"                   
[60] "SD_summary_all_binding_diff"                 
[61] "Domain"                                      
[62] "Phylum"                                      
[63] "Class"

**Individual predictors/correlations**

In [188]:
model.lm <- lm(SD_summary_all_info ~ ATG_start_percent,
                   data = data_comparative$data)
summary(model.lm)


Call:
lm(formula = SD_summary_all_info ~ ATG_start_percent, data = data_comparative$data)

Residuals:
    Min      1Q  Median      3Q     Max 
-1.4533 -0.5266 -0.1873  0.3387  2.4414 

Coefficients:
                  Estimate Std. Error t value Pr(>|t|)  
(Intercept)         0.4985     0.2497   1.997   0.0463 *
ATG_start_percent   0.6175     0.2987   2.068   0.0391 *
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.7414 on 611 degrees of freedom
Multiple R-squared:  0.006948,	Adjusted R-squared:  0.005323 
F-statistic: 4.275 on 1 and 611 DF,  p-value: 0.0391


In [ ]:
model.pgls <- pgls(SD_summary_all_info ~ ATG_start_percent,
                   data = data_comparative, lambda='ML',
                   bounds=list(lambda=c(1e-5,0.9999), kappa=c(1e-6,3), delta=c(1e-6,3)))
summary(model.pgls)

In [190]:
model.pgls <- pgls(SD_summary_all_info ~ 1,
                   data = data_comparative, lambda='ML')
summary(model.pgls)


Call:
pgls(formula = SD_summary_all_info ~ 1, data = data_comparative, 
    lambda = "ML")

Residuals:
     Min       1Q   Median       3Q      Max 
-2.26539 -0.41894 -0.05416  0.31945  2.94683 

Branch length transformations:

kappa  [Fix]  : 1.000
lambda [ ML]  : 0.992
   lower bound : 0.000, p = < 2.22e-16
   upper bound : 1.000, p = 0.001451
   95.0% CI   : (0.982, 0.998)
delta  [Fix]  : 1.000

Coefficients:
            Estimate Std. Error t value  Pr(>|t|)    
(Intercept)  1.08993    0.19632  5.5518 4.217e-08 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.6096 on 612 degrees of freedom
Multiple R-squared:     0,	Adjusted R-squared:     0 
F-statistic:   NaN on 0 and 612 DF,  p-value: NA 

## Correlation matrix between all methods

In [149]:
data_table <-read.csv('../Data/Pruned_df_Domain_Bacteria_10_2017.csv', sep='\t')
data_tree <-read.tree('../Data/Pruned_tree_Domain_Bacteria_10_2017.newick')

data_comparative <- comparative.data(phy = data_tree, data = data_table,
                                    names.col = X.Id, vcv = FALSE, na.omit = FALSE, warn.dropped = TRUE)
out_file_name <- '../Results/correlation_matrix_Bacteria.csv'



###To make it that higher values are associated with more codon bias in ribosome protein genes
data_table['NC_summary_info'] <- data_table['NC_summary_info']*-1
###To make it that higher values are associated with weaker aSD binding in ribosome protein genes
data_table['aSD_binding_summary'] <- data_table['aSD_binding_summary']*-1





########Limit the data?
# data_table <- data_table[grepl("Actinobacteria", data_table$Phylum), ]
# out_file_name <- '../Results/correlation_matrix_Actinobacteria.csv'
#
# data_table <- data_table[grepl("Firmicutes", data_table$Phylum), ]
# out_file_name <- '../Results/correlation_matrix_Firmicutes.csv'
#
# data_table <- data_table[grepl("Bacteroidetes", data_table$Phylum), ]
# out_file_name <- '../Results/correlation_matrix_Bacteroidetes.csv'
#
# data_table <- data_table[grepl("Proteobacteria", data_table$Phylum), ]
# out_file_name <- '../Results/correlation_matrix_Proteobacteria.csv'



data_comparative <- comparative.data(phy = data_tree, data = data_table,
                                    names.col = X.Id, vcv = TRUE, na.omit = FALSE, warn.dropped = TRUE)


In [150]:
metrics <- c('SD_summary_all_motif_diff', 'SD_summary_all_binding_diff', 'SD_summary_all_info',
             'ATG_start_percent', 'Mean_folding_diff', 'X16s_copies', 'tRNA_number', 
             'NC_summary_info', 'aSD_binding_summary')


**It's worth noting that I'm doing so many correlations below on a fairly large-ish dataset size that I fix lambda=1 for these figures**

In [151]:
rsq_table <- data.frame(matrix(ncol=9, nrow=9))
colnames(rsq_table) <- metrics
rownames(rsq_table) <- metrics

for(i in metrics){
    for(j in metrics){
        if(i != j){
#             print(paste(i, "~", j))
#             model.pgls <- pgls(as.formula(paste(i, "~", j)),
#                    data = data_comparative, lambda='ML',
#                    bounds=list(lambda=c(1e-5,0.9999), kappa=c(1e-6,3), delta=c(1e-6,3)))
            model.pgls <- pgls(as.formula(paste(i, "~", j)),
                               data = data_comparative, lambda=1.0)
#             model.pgls <- lm(as.formula(paste(i, "~", j)),
#                                data = data_comparative$data)
            print(paste(i, "~", j, "rsq", summary(model.pgls)$r.squared))
            anova_model <- anova(model.pgls)
            if (anova_model$'Pr(>F)'[1] < 0.01){
                if(summary(model.pgls)$coefficients[2,1] > 0.0){
                    rsq_table[i, j] <- sqrt(summary(model.pgls)$r.squared)
                }
                else if(summary(model.pgls)$coefficients[2,1] < 0.0){
                    rsq_table[i, j] <- -1* sqrt(summary(model.pgls)$r.squared)
                }
            }
        }
    }
}

[1] "SD_summary_all_motif_diff ~ SD_summary_all_binding_diff rsq 0.84890656238214"
[1] "SD_summary_all_motif_diff ~ SD_summary_all_info rsq 0.28745545974983"
[1] "SD_summary_all_motif_diff ~ ATG_start_percent rsq 0.179535382543885"
[1] "SD_summary_all_motif_diff ~ Mean_folding_diff rsq 0.0214551108858279"
[1] "SD_summary_all_motif_diff ~ X16s_copies rsq 0.0342932933670087"
[1] "SD_summary_all_motif_diff ~ tRNA_number rsq 0.0031835311784135"
[1] "SD_summary_all_motif_diff ~ NC_summary_info rsq 0.0187868255819828"
[1] "SD_summary_all_motif_diff ~ aSD_binding_summary rsq 0.0165834432408283"
[1] "SD_summary_all_binding_diff ~ SD_summary_all_motif_diff rsq 0.84890656238214"
[1] "SD_summary_all_binding_diff ~ SD_summary_all_info rsq 0.377616392499015"
[1] "SD_summary_all_binding_diff ~ ATG_start_percent rsq 0.185447088779336"
[1] "SD_summary_all_binding_diff ~ Mean_folding_diff rsq 0.00630541313539976"
[1] "SD_summary_all_binding_diff ~ X16s_copies rsq 0.0466176933021502"
[1] "SD_summary_all

In [152]:
rsq_table

,SD_summary_all_motif_diff,SD_summary_all_binding_diff,SD_summary_all_info,ATG_start_percent,Mean_folding_diff,X16s_copies,tRNA_number,NC_summary_info,aSD_binding_summary
SD_summary_all_motif_diff,NA,0.9213613,0.5361487,0.4237162,0.1464756,0.1851845,NA,0.137065,0.1287767
SD_summary_all_binding_diff,0.9213613,NA,0.614505,0.4306357,NA,0.2159113,NA,0.1217994,0.118973
SD_summary_all_info,0.5361487,0.614505,NA,0.3592393,0.1093719,0.2827616,0.1781241,0.1529194,0.204079
ATG_start_percent,0.4237162,0.4306357,0.3592393,NA,0.1575941,NA,NA,0.153973,0.1196348
Mean_folding_diff,0.1464756,NA,0.1093719,0.1575941,NA,NA,NA,NA,0.4238418
X16s_copies,0.1851845,0.2159113,0.2827616,NA,NA,NA,0.7183934,0.3614848,0.3082456
tRNA_number,NA,NA,0.1781241,NA,NA,0.7183934,NA,0.3649435,0.2856944
NC_summary_info,0.137065,0.1217994,0.1529194,0.153973,NA,0.3614848,0.3649435,NA,0.4637013
aSD_binding_summary,0.1287767,0.118973,0.204079,0.1196348,0.4238418,0.3082456,0.2856944,0.4637013,NA


In [153]:
write.table(rsq_table,file=out_file_name)

In [155]:
model.pgls <- pgls(SD_summary_all_info ~ SD_summary_all_info,
                   data = data_comparative, lambda=1.0)
summary(model.pgls)

Warning message:
In model.matrix.default(formula, m): the response appeared on the right-hand side and was droppedWarning message:
In model.matrix.default(formula, m): problem with term 1 in model.matrix: no columns are assigned


Call:
pgls(formula = SD_summary_all_info ~ SD_summary_all_info, data = data_comparative, 
    lambda = 1)

Residuals:
     Min       1Q   Median       3Q      Max 
-3.15203 -0.37799 -0.01662  0.34544  1.74031 

Branch length transformations:

kappa  [Fix]  : 1.000
lambda [Fix]  : 1.000
delta  [Fix]  : 1.000

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  1.08921    0.20543  5.3022  1.6e-07 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.6359 on 612 degrees of freedom
Multiple R-squared:     0,	Adjusted R-squared:     0 
F-statistic:   NaN on 0 and 612 DF,  p-value: NA 

# Temperature associations

In [156]:
data_table <-read.csv('../Data/Pruned_df_Domain_Bacteria_10_2017_WITH_PROTRAITS_TEMPERATURES.csv', sep='\t')
data_tree <-read.tree('../Data/Pruned_tree_Domain_Bacteria_10_2017.newick')

data_comparative <- comparative.data(phy = data_tree, data = data_table,
                                    names.col = X.Id, vcv = FALSE, na.omit = FALSE, warn.dropped = TRUE)

Warning message:
In comparative.data(phy = data_tree, data = data_table, names.col = X.Id, : Data dropped in compiling comparative data object

In [161]:
model.pgls <- pgls(SD_summary_all_info ~ temperature,
                   data = data_comparative, lambda='ML')
summary(model.pgls)


Call:
pgls(formula = SD_summary_all_info ~ temperature, data = data_comparative, 
    lambda = "ML")

Residuals:
     Min       1Q   Median       3Q      Max 
-2.77057 -0.35868 -0.02525  0.34357  1.62590 

Branch length transformations:

kappa  [Fix]  : 1.000
lambda [ ML]  : 0.990
   lower bound : 0.000, p = < 2.22e-16
   upper bound : 1.000, p = 0.0008019
   95.0% CI   : (0.978, 0.997)
delta  [Fix]  : 1.000

Coefficients:
                        Estimate Std. Error t value  Pr(>|t|)    
(Intercept)             1.128535   0.173866  6.4908 2.136e-10 ***
temperaturethermophiles 0.250797   0.082964  3.0230  0.002637 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.6142 on 479 degrees of freedom
Multiple R-squared: 0.01872,	Adjusted R-squared: 0.01667 
F-statistic: 9.138 on 1 and 479 DF,  p-value: 0.002637 

**This didn't make it into the paper but increases my confidence 1000-fold in the temperature result**
First, note that all predictors combined do a decent job at predicting deltaI

In [166]:
model.pgls <- pgls(SD_summary_all_info ~ ATG_start_percent + Mean_folding_diff + X16s_copies +
                   tRNA_number + NC_summary_info + aSD_binding_summary,
                   data = data_comparative, lambda='ML')
summary(model.pgls)


Call:
pgls(formula = SD_summary_all_info ~ ATG_start_percent + Mean_folding_diff + 
    X16s_copies + tRNA_number + NC_summary_info + aSD_binding_summary, 
    data = data_comparative, lambda = "ML")

Residuals:
     Min       1Q   Median       3Q      Max 
-2.16895 -0.30757 -0.00513  0.38972  2.13753 

Branch length transformations:

kappa  [Fix]  : 1.000
lambda [ ML]  : 1.000
   lower bound : 0.000, p = < 2.22e-16
   upper bound : 1.000, p = 0.90888
   95.0% CI   : (0.990, NA)
delta  [Fix]  : 1.000

Coefficients:
                       Estimate  Std. Error t value  Pr(>|t|)    
(Intercept)         -0.38878759  0.26494774 -1.4674   0.14293    
ATG_start_percent    1.93313303  0.25400756  7.6105 1.485e-13 ***
Mean_folding_diff   -0.00279239  0.01055128 -0.2646   0.79139    
X16s_copies          0.05546077  0.01211544  4.5777 6.014e-06 ***
tRNA_number         -0.00076625  0.00158998 -0.4819   0.63008    
NC_summary_info      0.63168046  0.54003281  1.1697   0.24271    
aSD_binding_summ

**Now, after accounting for all that, adding in temperature as a categorical variable is highly significant**

In [167]:
model.pgls <- pgls(SD_summary_all_info ~ ATG_start_percent + Mean_folding_diff + X16s_copies +
                   tRNA_number + NC_summary_info + aSD_binding_summary + temperature,
                   data = data_comparative, lambda='ML')
summary(model.pgls)


Call:
pgls(formula = SD_summary_all_info ~ ATG_start_percent + Mean_folding_diff + 
    X16s_copies + tRNA_number + NC_summary_info + aSD_binding_summary + 
    temperature, data = data_comparative, lambda = "ML")

Residuals:
     Min       1Q   Median       3Q      Max 
-2.01433 -0.33283 -0.00388  0.39543  1.92982 

Branch length transformations:

kappa  [Fix]  : 1.000
lambda [ ML]  : 1.000
   lower bound : 0.000, p = < 2.22e-16
   upper bound : 1.000, p = 1    
   95.0% CI   : (0.989, NA)
delta  [Fix]  : 1.000

Coefficients:
                           Estimate  Std. Error t value  Pr(>|t|)    
(Intercept)             -0.72631863  0.26564834 -2.7341  0.006489 ** 
ATG_start_percent        2.10358063  0.24898956  8.4485 4.441e-16 ***
Mean_folding_diff       -0.00071812  0.01026660 -0.0699  0.944265    
X16s_copies              0.05682279  0.01177930  4.8240 1.901e-06 ***
tRNA_number             -0.00116589  0.00154554 -0.7544  0.451012    
NC_summary_info          0.36533668  0.5278403

**Other environmental features**

In [169]:
data_table <-read.csv('../Data/Pruned_df_Domain_Bacteria_10_2017_WITH_PROTRAITS_HABITAT.csv', sep='\t')
data_tree <-read.tree('../Data/Pruned_tree_Domain_Bacteria_10_2017.newick')

data_comparative <- comparative.data(phy = data_tree, data = data_table,
                                    names.col = X.Id, vcv = FALSE, na.omit = FALSE, warn.dropped = TRUE)

Warning message:
In comparative.data(phy = data_tree, data = data_table, names.col = X.Id, : Data dropped in compiling comparative data object

In [170]:
model.pgls <- pgls(SD_summary_all_info ~ habitat,
                   data = data_comparative, lambda='ML')
summary(model.pgls)


Call:
pgls(formula = SD_summary_all_info ~ habitat, data = data_comparative, 
    lambda = "ML")

Residuals:
     Min       1Q   Median       3Q      Max 
-2.40807 -0.39541 -0.01356  0.32012  2.59317 

Branch length transformations:

kappa  [Fix]  : 1.000
lambda [ ML]  : 0.989
   lower bound : 0.000, p = < 2.22e-16
   upper bound : 1.000, p = 0.00049903
   95.0% CI   : (0.974, 0.997)
delta  [Fix]  : 1.000

Coefficients:
             Estimate Std. Error t value  Pr(>|t|)    
(Intercept) 1.2727295  0.1701654  7.4794 3.793e-13 ***
habitathost 0.0059938  0.0530338  0.1130    0.9101    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.621 on 462 degrees of freedom
Multiple R-squared: 2.765e-05,	Adjusted R-squared: -0.002137 
F-statistic: 0.01277 on 1 and 462 DF,  p-value: 0.9101 

In [171]:
data_table <-read.csv('../Data/Pruned_df_Domain_Bacteria_10_2017_WITH_PROTRAITS_PATHOGEN.csv', sep='\t')
data_tree <-read.tree('../Data/Pruned_tree_Domain_Bacteria_10_2017.newick')

data_comparative <- comparative.data(phy = data_tree, data = data_table,
                                    names.col = X.Id, vcv = FALSE, na.omit = FALSE, warn.dropped = TRUE)

Warning message:
In comparative.data(phy = data_tree, data = data_table, names.col = X.Id, : Data dropped in compiling comparative data object

In [172]:
model.pgls <- pgls(SD_summary_all_info ~ pathogen,
                   data = data_comparative, lambda='ML')
summary(model.pgls)


Call:
pgls(formula = SD_summary_all_info ~ pathogen, data = data_comparative, 
    lambda = "ML")

Residuals:
     Min       1Q   Median       3Q      Max 
-2.51328 -0.32960 -0.01265  0.39848  2.18707 

Branch length transformations:

kappa  [Fix]  : 1.000
lambda [ ML]  : 0.990
   lower bound : 0.000, p = < 2.22e-16
   upper bound : 1.000, p = 0.00089749
   95.0% CI   : (0.978, 0.997)
delta  [Fix]  : 1.000

Coefficients:
            Estimate Std. Error t value  Pr(>|t|)    
(Intercept) 1.280283   0.169569  7.5502 2.096e-13 ***
pathogenYes 0.029436   0.038233  0.7699    0.4417    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.6191 on 497 degrees of freedom
Multiple R-squared: 0.001191,	Adjusted R-squared: -0.0008184 
F-statistic: 0.5928 on 1 and 497 DF,  p-value: 0.4417 

# No association between delta I and genome length or CDS number

In [173]:
data_table <-read.csv('../Data/Pruned_df_Domain_Bacteria_10_2017.csv', sep='\t')
data_tree <-read.tree('../Data/Pruned_tree_Domain_Bacteria_10_2017.newick')

data_comparative <- comparative.data(phy = data_tree, data = data_table,
                                    names.col = X.Id, vcv = FALSE, na.omit = FALSE, warn.dropped = TRUE)

In [175]:
model.pgls <- pgls(SD_summary_all_info ~ CDS_number,
                   data = data_comparative, lambda='ML')
summary(model.pgls)


Call:
pgls(formula = SD_summary_all_info ~ CDS_number, data = data_comparative, 
    lambda = "ML")

Residuals:
     Min       1Q   Median       3Q      Max 
-2.25840 -0.41737 -0.03949  0.32958  2.93526 

Branch length transformations:

kappa  [Fix]  : 1.000
lambda [ ML]  : 0.992
   lower bound : 0.000, p = < 2.22e-16
   upper bound : 1.000, p = 0.0017542
   95.0% CI   : (0.982, 0.998)
delta  [Fix]  : 1.000

Coefficients:
              Estimate Std. Error t value  Pr(>|t|)    
(Intercept) 1.0751e+00 1.9814e-01  5.4260 8.319e-08 ***
CDS_number  6.3524e-06 1.0501e-05  0.6049    0.5455    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.6105 on 611 degrees of freedom
Multiple R-squared: 0.0005985,	Adjusted R-squared: -0.001037 
F-statistic: 0.3659 on 1 and 611 DF,  p-value: 0.5455 

In [176]:
model.pgls <- pgls(SD_summary_all_info ~ Genome_length,
                   data = data_comparative, lambda='ML')
summary(model.pgls)


Call:
pgls(formula = SD_summary_all_info ~ Genome_length, data = data_comparative, 
    lambda = "ML")

Residuals:
     Min       1Q   Median       3Q      Max 
-2.25322 -0.41461 -0.03234  0.33097  2.94066 

Branch length transformations:

kappa  [Fix]  : 1.000
lambda [ ML]  : 0.992
   lower bound : 0.000, p = < 2.22e-16
   upper bound : 1.000, p = 0.0016337
   95.0% CI   : (0.982, 0.998)
delta  [Fix]  : 1.000

Coefficients:
                Estimate Std. Error t value  Pr(>|t|)    
(Intercept)   1.0855e+00 1.9791e-01  5.4845 6.077e-08 ***
Genome_length 1.7181e-09 8.9371e-09  0.1922    0.8476    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.6103 on 611 degrees of freedom
Multiple R-squared: 6.048e-05,	Adjusted R-squared: -0.001576 
F-statistic: 0.03696 on 1 and 611 DF,  p-value: 0.8476 

# Scratch

In [178]:
model.pgls <- pgls(SD_summary_all_info ~ SD_summary_ribo_info,
                   data = data_comparative, lambda='ML')
summary(model.pgls)


Call:
pgls(formula = SD_summary_all_info ~ SD_summary_ribo_info, data = data_comparative, 
    lambda = "ML")

Residuals:
     Min       1Q   Median       3Q      Max 
-1.42185 -0.30897  0.00267  0.31792  1.60285 

Branch length transformations:

kappa  [Fix]  : 1.000
lambda [ ML]  : 0.963
   lower bound : 0.000, p = < 2.22e-16
   upper bound : 1.000, p = 5.5689e-12
   95.0% CI   : (0.936, 0.981)
delta  [Fix]  : 1.000

Coefficients:
                     Estimate Std. Error t value  Pr(>|t|)    
(Intercept)          0.417030   0.158016  2.6392  0.008523 ** 
SD_summary_ribo_info 0.326044   0.020361 16.0135 < 2.2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.4783 on 611 degrees of freedom
Multiple R-squared: 0.2956,	Adjusted R-squared: 0.2945 
F-statistic: 256.4 on 1 and 611 DF,  p-value: < 2.2e-16 